## (1) Load Processed Data in the Paper

Currently, we support norman / adamson / dixit.

In [1]:
import os
os.chdir("/gpfs/home/asun/jin_lab/GEARS/demo/")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

from gears import PertData

/gpfs/group/jin/asun/GEARS/demo


/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/utils.py:434: Futur

In [18]:
import sys
sys.path.append(os.path.abspath('..'))

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.load(data_name = 'norman') # specific dataset name
pert_data.prepare_split(split = 'simulation', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['RHOXF2BB+ctrl' 'LYL1+IER5L' 'ctrl+IER5L' 'KIAA1804+ctrl' 'IER5L+ctrl'
 'RHOXF2BB+ZBTB25' 'RHOXF2BB+SET']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Simulation split test composition:
combo_seen0:9
combo_seen1:43
combo_seen2:19
unseen_single:36
Done!
Creating dataloaders....
Done!


here1


In [20]:
pert_data.adata

View of AnnData object with n_obs × n_vars = 89357 × 5045
    obs: 'condition', 'cell_type', 'dose_val', 'control', 'condition_name'
    var: 'gene_name'
    uns: 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'top_non_zero_de_20'
    layers: 'counts'

In [21]:
pert_data.adata.obs["condition"].value_counts()

condition
ctrl             7353
CEBPE+RUNX1T1    1030
KLF1+ctrl         997
TBX3+TBX2         969
SLC4A1+ctrl       853
                 ... 
PLK4+STIL          66
SAMD1+TGFBR2       59
JUN+CEBPB          52
CBL+UBASH3A        50
C3orf72+FOXL2      49
Name: count, Length: 277, dtype: int64

In [22]:
pert_data.adata.obs["cell_type"].value_counts()

cell_type
A549    89357
Name: count, dtype: int64

In [6]:
pert_data.set2conditions

{'test': ['CBL+PTPN9',
  'DUSP9+ctrl',
  'MAP2K6+SPI1',
  'UBASH3B+PTPN12',
  'BCORL1+ctrl',
  'MEIS1+ctrl',
  'CBL+ctrl',
  'KLF1+FOXA1',
  'TBX3+TBX2',
  'SLC4A1+ctrl',
  'DUSP9+MAPK1',
  'COL2A1+ctrl',
  'CEBPE+KLF1',
  'UBASH3B+OSR2',
  'UBASH3B+ZBTB25',
  'DUSP9+ETS2',
  'ZNF318+FOXL2',
  'UBASH3B+PTPN9',
  'S1PR2+ctrl',
  'CELF2+ctrl',
  'JUN+CEBPA',
  'CDKN1A+ctrl',
  'ctrl+MEIS1',
  'MAPK1+PRTG',
  'MAP2K3+IKZF3',
  'KLF1+COL2A1',
  'PTPN12+OSR2',
  'ETS2+CEBPE',
  'POU3F2+FOXL2',
  'DUSP9+PRTG',
  'CKS1B+ctrl',
  'BCL2L11+TGFBR2',
  'AHR+KLF1',
  'CEBPB+CEBPA',
  'PRTG+ctrl',
  'ETS2+CNN1',
  'C3orf72+ctrl',
  'CNN1+MAPK1',
  'FOXL2+MEIS1',
  'FOXL2+ctrl',
  'FOSB+CEBPE',
  'PTPN12+PTPN9',
  'FOSB+CEBPB',
  'ctrl+CDKN1A',
  'BCL2L11+BAK1',
  'FOXA3+HOXB9',
  'ARID1A+ctrl',
  'ctrl+COL2A1',
  'CEBPE+CNN1',
  'ZC3HAV1+HOXC13',
  'CBL+CNN1',
  'ZBTB10+PTPN12',
  'CBL+UBASH3B',
  'CEBPB+PTPN12',
  'BCL2L11+ctrl',
  'OSR2+ctrl',
  'ctrl+SPI1',
  'CEBPB+MAPK1',
  'ETS2+MAPK1',
  'DU

In [14]:
for key, value in pert_data.set2conditions.items():
    print(key)
    print(len(value))

test
5
train
12
val
2


## (2) Create your own Perturb-Seq data
Prepare a scanpy adata object with 
1. `adata.obs` dataframe has `condition` and `cell_type` columns, where `condition` is the perturbation name for each cell. Control cells have condition format of `ctrl`, single perturbation has condition format of `A+ctrl` or `ctrl+A`, combination perturbation has condition format of `A+B`.
2. `adata.var` dataframe has `gene_name` column, where each gene name is the gene symbol.
3. `adata.X` stores the post-perturbed gene expression. 

Here an example using dixit 2016 dataset.

In [3]:
adata_sim = sc.read_h5ad('simulated_adata.h5ad')
adata_sim

AnnData object with n_obs × n_vars = 25738 × 5000
    obs: 'condition', 'perturbation'
    var: 'variable_gene', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'

In [7]:
adata_sim.obs = adata_sim.obs.rename(columns={"condition": "target_gene"})


In [13]:
adata_sim.obs['target_gene'] = adata_sim.obs['target_gene'].replace('ctrl', 'non-targeting')


In [14]:
adata_sim.write("/gpfs/home/asun/jin_lab/get/cell-eval/runs/simulated_adata.h5ad")

In [9]:
adata_real = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/GEARS/demo/data/boli/perturb_processed.h5ad')
adata_real

AnnData object with n_obs × n_vars = 35527 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident', 'Assign_clean', 'condition', 'cell_type', 'dose_val', 'control', 'condition_name'
    var: 'variable_gene', 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'non_dropout_gene_idx', 'non_zeros_gene_idx', 'rank_genes_groups_cov_all', 'top_non_dropout_de_20', 'top_non_zero_de_20'

In [ ]:
adata_real.obs = adata_real.obs.rename(columns={"condition": "target_gene"})


In [12]:
exclude_genes = ['TRIO+ctrl', 'DAB1+ctrl', 'CX3CL1+ctrl']
# Filter out those rows
adata_real_filtered = adata_real[~adata_real.obs['target_gene'].isin(exclude_genes)].copy()
adata_real_filtered.obs['target_gene'] = adata_real_filtered.obs['target_gene'].replace('ctrl', 'non-targeting')
adata_real_filtered.write("/gpfs/home/asun/jin_lab/get/cell-eval/runs/real_adata.h5ad")

In [4]:
gene_list = adata_sim.var_names.to_numpy()

In [5]:
gene_list

array(['4', '5', '7', ..., '32205', '32207', '32261'],
      shape=(5000,), dtype=object)

In [6]:
import pandas as pd

pd.Series(gene_list).to_csv("gene_list.csv", index=False, header=False)

## Processing Xinhe

In [3]:
import scanpy as sc
adata = sc.read_h5ad('/gpfs/home/juntongy/gloria/my_gears/GEARS/processed.h5ad')
adata

/gpfs/home/asun/miniforge3/envs/gears/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 14511 × 32285
    obs: 'sample', 'class_label', 'class_name', 'class_bootstrapping_probability', 'subclass_label', 'subclass_name', 'subclass_bootstrapping_probability', 'supertype_label', 'supertype_name', 'supertype_bootstrapping_probability', 'cluster_label', 'cluster_name', 'cluster_alias', 'cluster_bootstrapping_probability', 'cell_type', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_CRISPR', 'nFeature_CRISPR', 'predicted.class.score', 'predicted.class', 'predicted.cluster.score', 'predicted.cluster', 'predicted.subclass.score', 'predicted.subclass', 'predicted.cross_species_cluster.score', 'predicted.cross_species_cluster', 'mapping.score', 'unintegrated_clusters', 'seurat_clusters', 'condition'
    var: 'gene_name'
    obsm: 'X_ref.umap', 'X_umap.unintegrated'
    layers: 'CRISPR'

In [3]:
adata.var.head(2)

,variable_gene
Xkr4,True
Gm1992,False


In [4]:
# Store the current index (gene names) into a new column called 'gene_name'
adata.var['gene_name'] = adata.var.index

# Optionally reset the index if you want to replace it with default integers
adata.var.reset_index(drop=True, inplace=True)

In [5]:
adata.obs['pertgene'].value_counts()

pertgene
ctrl                          2770
Tcf4                          2276
Foxg1                         1982
Nr2f1                         1885
Tbr1                          1448
                              ... 
Nr2f1_1,Tcf4_2,Foxg1_1,GFP       1
Nr2f1_1,Tcf4_3                   1
Nr2f1_2,GFP,Foxg1_1              1
Nr2f1_2,NonTarget2               1
Nr2f1_3,Foxg1_1,Foxg1_2          1
Name: count, Length: 283, dtype: int64

In [6]:
adata = adata[~adata.obs['pertgene'].str.contains(',')]

In [7]:
adata.obs['pertgene'].value_counts()

pertgene
ctrl     2770
Tcf4     2276
Foxg1    1982
Nr2f1    1885
Tbr1     1448
Name: count, dtype: int64

In [8]:
adata.obs['condition'] = adata.obs['pertgene'].apply(
    lambda x: x if x == 'ctrl' else f"{x.split('+')[0].upper()}+ctrl"
)

In [9]:
adata.obs['condition'].value_counts()

condition
ctrl          2770
TCF4+ctrl     2276
FOXG1+ctrl    1982
NR2F1+ctrl    1885
TBR1+ctrl     1448
Name: count, dtype: int64

In [10]:
adata.obs['CellType'].value_counts()

CellType
Excit_L6CT_CTX      2918
Excit_L5IT          1103
CR                  1029
Excit_Upper          983
Excit_L6IT           948
Excit_L5NP_CTX       899
Excit_L2 IT ENTl     498
Excit_L5 PT CTX      470
Inhib_Lhx6+Sst-      435
Inhib_Sst            387
Excit_L6b/CT ENT     247
Excit_L6b CTX        226
Excit_Car3           131
Inhib_Id2             87
Name: count, dtype: int64

In [11]:
adata.obs.rename(columns={'CellType': 'cell_type'}, inplace=True)

In [12]:
adata.obs['cell_type'].value_counts()

cell_type
Excit_L6CT_CTX      2918
Excit_L5IT          1103
CR                  1029
Excit_Upper          983
Excit_L6IT           948
Excit_L5NP_CTX       899
Excit_L2 IT ENTl     498
Excit_L5 PT CTX      470
Inhib_Lhx6+Sst-      435
Inhib_Sst            387
Excit_L6b/CT ENT     247
Excit_L6b CTX        226
Excit_Car3           131
Inhib_Id2             87
Name: count, dtype: int64

In [13]:
# test to see if the cell type split is the issue
adata.obs['cell_type'] = 'Neuron'

In [14]:
adata.var

,variable_gene,gene_name
0,True,Xkr4
1,False,Gm1992
2,False,Gm19938
3,False,Gm37381
4,False,Rp1
...,...,...
32280,False,AC124606.1
32281,False,AC133095.2
32282,False,AC133095.1
32283,False,AC234645.1


In [15]:
adata.obs.head(2)

,Name,orig.ident,nCount_RNA,nFeature_RNA,scds,cxds,bcds,RNA_snn_res.0.8,seurat_clusters,nCount_refAssay,...,Max,MaxVal,SecMaxVal,demuxHC,demuxCR,demuxDS,assignDS,pertgene,active_ident,condition
Ch1_AAACCTGTCAGGTTCA-1,Ch1_Nr2f1_3,Ch1,17773,5382,0.595517,714.451233,0.438533,6,6,17773,...,Nr2f13,176,8,singlet,singlet,singlet,Nr2f1_3,Nr2f1,6,NR2F1+ctrl
Ch1_AAACCTGTCCTTTACA-1,Ch1_NonTarget1,Ch1,11213,4506,0.385882,697.850370,0.233780,4,4,11213,...,NonTarget1,112,4,singlet,singlet,singlet,NonTarget1,ctrl,4,ctrl


### Suggested normalization

For raw count data we recommend the following normalization and subsetting to the top 5000 most variable genes

In [16]:
import pickle

with open("/gpfs/home/asun/jin_lab/get/GEARS/demo/perturb_indices.pkl", "rb") as f:
    perturb_indices = pickle.load(f)

In [17]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
##sc.pp.highly_variable_genes(adata,n_top_genes=5000, subset=True)


In [18]:
adata = adata[:, perturb_indices].copy()

In [19]:
adata


AnnData object with n_obs × n_vars = 10361 × 2340
    obs: 'Name', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'scds', 'cxds', 'bcds', 'RNA_snn_res.0.8', 'seurat_clusters', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'feature_call', 'num_features', 'num_umis', 'nCount_Crispr', 'nFeature_Crispr', 'RNA_snn_res.1.5', 'lowQC', 'cell_type', 'Perc', 'CBC', 'antisense', 'intergenic', 'intronic', 'exonic', 'multi', 'unmapped', 'highConf', 'polyA', 'TSO', 'spliced', 'percent_qual_cbc', 'percent_qual_umi', 'nUMI', 'UTR', 'total', 'Rem', 'CellType_Gen', 'Mito', 'demux_type', 'assignment', 'Keep', 'Max', 'MaxVal', 'SecMaxVal', 'demuxHC', 'demuxCR', 'demuxDS', 'assignDS', 'pertgene', 'active_ident', 'condition'
    var: 'variable_gene', 'gene_name'
    uns: 'RNA_snn_res.0.8_colors', 'active_ident_colors', 'seurat_clusters_colors', 'log1p'
    obsm: 'X_gbm', 'X_pca', 'X_umap'
    layers: 'raw_count'

### Create dataloader

GEARS will take it from here. The new data processing takes around 15 minutes for 5K genes and 100K cells. 

In [20]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.new_data_process(dataset_name = 'zheng_GET', adata = adata) # specific dataset name and adata object
pert_data.load(data_path = './data/zheng_get') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:29<00:00, 29.81s/it]
Done!
Saving new dataset pyg object at ./data/zheng_get/data_pyg/cell_graphs.pkl
Done!


ValueError: data attribute is either norman, adamson, dixit replogle_k562 or replogle_rpe1 or a path to an h5ad file

Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
  0%|          | 0/5 [00:00<?, ?it/s]

In [21]:
pert_data.load(data_path = './data/zheng_get') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
[]
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at ./data/zheng_get/splits/zheng_get_simulation_single_1_0.75.pkl
Done!
Creating dataloaders....
Done!


# processing boli

In [6]:
import scanpy as sc
adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/raw_data/seur.cur.h5ad')
adata

AnnData object with n_obs × n_vars = 35527 × 32285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident'
    var: 'variable_gene'

In [28]:
print(adata.to_df().iloc[:10, :10])


                                       Xkr4  Gm1992  Gm19938  Gm37381  Rp1  \
batch1_samp1_chan1_AAACCTGAGACAAAGG-1   5.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGAGGCCGAAT-1  15.0     0.0      0.0      0.0  1.0   
batch1_samp1_chan1_AAACCTGCAACTGGCC-1  15.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGCACGGTAAG-1   1.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGCACTATCTT-1   3.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGCAGGGCATA-1   3.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGCATAACCTG-1   4.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGGTGCTTCTC-1   1.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGGTGTCGCTG-1   3.0     0.0      0.0      0.0  0.0   
batch1_samp1_chan1_AAACCTGGTTGTCGCG-1   1.0     1.0      0.0      0.0  0.0   

                                       Sox17  Gm37587  Gm37323  Mrpl15  Lypla1  
batch1_samp1_chan1_AAACCTGAGACAAAGG-1    0.0      0.0      0

In [34]:
import numpy as np

gene_name = 'Tafa1'  # replace with your gene of interest
cell_type = 'L6 CT CTX'  # your cell type of interest

# Get gene index
gene_idx = adata.var_names.get_loc(gene_name)

# Filter cells by condition AND cell type
perturb_cells = adata[(adata.obs['Assign'] == 'Tbr1_0') & (adata.obs['predicted.subclass'] == cell_type)]
control_cells = adata[(adata.obs['Assign'] == 'NT_0') & (adata.obs['predicted.subclass'] == cell_type)]

# Calculate mean expression for the gene in each group
mean_perturb = perturb_cells.X[:, gene_idx].mean()
mean_control = control_cells.X[:, gene_idx].mean()

# Handle sparse matrix if needed
if hasattr(mean_perturb, "toarray"):
    mean_perturb = mean_perturb.toarray().ravel()[0]
    mean_control = mean_control.toarray().ravel()[0]

# Pseudocount to avoid div by zero
pseudocount = 1e-9
log2fc = np.log2((mean_perturb + pseudocount) / (mean_control + pseudocount))

print(f'Log2FC for {gene_name} in {cell_type}: {log2fc}')

Log2FC for Tafa1 in L6 CT CTX: 3.310690559195603


In [36]:
adata.var.head(2)

,variable_gene
Xkr4,False
Gm1992,False


In [48]:
# Store the current index (gene names) into a new column called 'gene_name'
adata.var['gene_name'] = adata.var.index

# Optionally reset the index if you want to replace it with default integers
adata.var.reset_index(drop=True, inplace=True)

In [8]:
# Show the unique cell types
adata.obs['Assign'].value_counts()

Assign
Bcl11b_0    5367
Trio_0      4274
Dab1_0      4123
Tbr1_0      3466
NT_0        2735
Rb1cc1_0    2534
Satb2_0     2320
Herc1_0     2165
NT_1        1979
Ank3_0      1763
Xpo7_0      1602
Cx3cl1_0    1392
Cul1_0      1297
ST_0         307
ST_1         203
Name: count, dtype: int64

In [9]:
def clean_assign(val):
    if val in {'NT_0', 'NT_1', 'ST_0', 'ST_1'}:
        return 'ctrl'
    else:
        return val.replace('_0', '').capitalize()

adata.obs['Assign_clean'] = adata.obs['Assign'].apply(clean_assign)

In [50]:
def clean_assign(val):
    if val in {'NT_0'}:
        return 'ctrl'
    else:
        return val.replace('_0', '').capitalize()

adata.obs['Assign_clean'] = adata.obs['Assign'].apply(clean_assign)

to_remove = ['Nt_1', 'St', 'St_1']
adata = adata[~adata.obs['Assign_clean'].isin(to_remove)].copy()

In [10]:
# Show the unique cell types
adata.obs['Assign_clean'].value_counts()

Assign_clean
Bcl11b    5367
ctrl      5224
Trio      4274
Dab1      4123
Tbr1      3466
Rb1cc1    2534
Satb2     2320
Herc1     2165
Ank3      1763
Xpo7      1602
Cx3cl1    1392
Cul1      1297
Name: count, dtype: int64

In [52]:
adata.obs['condition'] = adata.obs['Assign_clean'].apply(
    lambda x: x if x == 'ctrl' else f"{x.split('+')[0].upper()}+ctrl"
)

In [4]:
adata.obs['condition'].value_counts()

condition
ctrl           5379
BCL11B+ctrl    2561
XPO7+ctrl      1376
RB1CC1+ctrl    1332
TBR1+ctrl      1084
HERC1+ctrl      801
CUL1+ctrl       776
ANK3+ctrl       685
SATB2+ctrl      517
Name: count, dtype: int64

In [14]:
adata.obs.head(2)

,orig.ident,nCount_RNA,nFeature_RNA,seurat_clusters,Assign,scds,cxds,bcds,Sample,nCount_refAssay,...,predicted.subclass,CT,mito,BioSamp,CT2,ForPlot,Remove,active_ident,Assign_clean,condition
batch1_samp1_chan1_AAACCTGAGACAAAGG-1,batch1,6831,2953,7,Bcl11b_0,1.323674,28996.749458,0.880423,batch1_samp1_chan1,6831,...,L5 PT CTX,L6 CT CTX,0.029278,batch1_samp1,030 L6 CT CTX Glut,batch1_samp1_Bcl11b_0,False,7,Bcl11b,BCL11B+ctrl
batch1_samp1_chan1_AAACCTGAGGCCGAAT-1,batch1,6016,2767,21,Bcl11b_0,0.487555,9829.195327,0.337222,batch1_samp1_chan1,6016,...,L6b CTX,L6b CTX,0.016622,batch1_samp1,029 L6b CTX Glut,batch1_samp1_Bcl11b_0,False,21,Bcl11b,BCL11B+ctrl


In [11]:
print(adata.obs['predicted.subclass'].value_counts())

predicted.subclass
L6 CT CTX        8508
L4/5 IT CTX      4029
L5 PT CTX        4004
L2/3 IT CTX-1    3379
L5 NP CTX        2954
L6 IT CTX        2712
L5 IT CTX        2077
CT SUB           1795
Pvalb            1427
L2/3 IT CTX-2    1192
Sst               909
NP SUB            498
L6b CTX           396
L3 RSP-ACA        386
L2/3 IT PPP       382
L5 IT TPE-ENT     179
L5 PPP            165
Vip               128
Lamp5             122
Oligo              53
Sncg               53
SUB-ProS           33
Car3               33
NP PPP             31
Astro              31
CR                 15
Micro-PVM           9
L2 IT RHP           7
Endo                6
SMC-Peri            6
VLMC                3
L6b/CT ENT          1
L2 IT ENTl          1
L6 IT ENTl          1
V3d                 1
CA2                 1
Name: count, dtype: int64


In [2]:
print(adata.obs['predicted.subclass'].nunique())

36


In [56]:
to_keep = ['L6 CT CTX']
adata = adata[adata.obs['predicted.subclass'].isin(to_keep)].copy()

In [19]:
adata

AnnData object with n_obs × n_vars = 7860 × 32285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident', 'Assign_clean', 'condition'
    var: 'variable_gene', 'gene_name'

In [20]:
adata.obs['cell_type'] = 'L6 CT CTX'

In [29]:
adata.obs['cell_type'] = 'Neuron'

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata,n_top_genes=5000, subset=True)

In [30]:
adata

AnnData object with n_obs × n_vars = 35527 × 32285
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'seurat_clusters', 'Assign', 'scds', 'cxds', 'bcds', 'Sample', 'nCount_refAssay', 'nFeature_refAssay', 'predicted.subclass.score', 'predicted.subclass', 'CT', 'mito', 'BioSamp', 'CT2', 'ForPlot', 'Remove', 'active_ident'
    var: 'variable_gene'
    uns: 'log1p'

In [23]:
import sys
sys.path.append('../')

from gears import PertData

pert_data = PertData('./data') # specific saved folder
pert_data.new_data_process(dataset_name = 'boli_l6_ct', adata = adata) # specific dataset name and adata object
pert_data.load(data_path = './data/boli_l6_ct') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...


Found local copy...
Creating pyg object for each cell in the data...
Creating dataset file...
  0%|          | 0/12 [00:00<?, ?it/s]

TRIO+ctrl


 58%|█████▊    | 7/12 [00:33<00:19,  3.84s/it]

DAB1+ctrl


 83%|████████▎ | 10/12 [00:48<00:08,  4.46s/it]

CX3CL1+ctrl


100%|██████████| 12/12 [00:56<00:00,  4.68s/it]
Done!
Saving new dataset pyg object at ./data/boli_l6_ct/data_pyg/cell_graphs.pkl
Done!
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TRIO+ctrl' 'DAB1+ctrl' 'CX3CL1+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at ./data/boli_l6_ct/splits/boli_l6_ct_simulation_single_1_0.75.pkl
Done!
Creating dataloaders....
Done!


In [ ]:
import os
os.chdir("/gpfs/home/asun/jin_lab/get/GEARS/demo/")
print(os.getcwd())

import sys
sys.path.append(os.path.abspath('..'))

from gears import PertData

/gpfs/group/jin/asun/get/GEARS/demo


In [ ]:
pert_data = PertData('./data') # specific saved folder
#pert_data.new_data_process(dataset_name = 'boli', adata = adata) # specific dataset name and adata object
pert_data.load(data_path = './data/boli') # load the processed data, the path is saved folder + dataset_name
pert_data.prepare_split(split = 'simulation_single', seed = 1) # get data split with seed
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader

Found local copy...
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['TRIO+ctrl' 'DAB1+ctrl' 'CX3CL1+ctrl']
Local copy of pyg dataset is detected. Loading...
Done!
Local copy of split is detected. Loading...
Done!
Creating dataloaders....
Done!


here1
dict_keys(['test', 'train', 'val'])
simulation_single


# checking dense vs sparse


In [19]:
boli_adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/GEARS/demo/data/boli/perturb_processed.h5ad')


In [21]:
type(boli_adata.X)               # Is it <class 'numpy.ndarray'> or <class 'scipy.sparse.csr.csr_matrix'>?
adata.X[:5]                 # Try printing a few rows — is it sparse-looking?


scipy.sparse._csr.csr_matrix

In [20]:
norman_adata = sc.read_h5ad('/gpfs/home/asun/jin_lab/get/GEARS/demo/data/norman/perturb_processed.h5ad')


In [22]:
type(norman_adata.X)               # Is it <class 'numpy.ndarray'> or <class 'scipy.sparse.csr.csr_matrix'>?


scipy.sparse._csr.csr_matrix